In [94]:
# Convert this notebook to a Python script and save as a new file
!jupyter nbconvert --to script model_5.ipynb

[NbConvertApp] Converting notebook model_5.ipynb to script
[NbConvertApp] Writing 2340 bytes to model_5.py


In [15]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import glob
import string

In [93]:
charlist = string.ascii_lowercase + ' \n' + string.digits
print(charlist)

abcdefghijklmnopqrstuvwxyz 
0123456789


In [78]:
dataset = []
labels = []

# TODO: move into function
# Loop through categories
for category, value in [('neg', 0), ('pos', 1)]:
#     Get list of files in dataset and truncate
    filenames = glob.glob(f'../train/{category}/*.txt')[:100]
    for i, f in enumerate(filenames):
        with open(f) as textfile:
            p = string.punctuation
#             Get text data and remove punctuation
            content = textfile.read().translate(str.maketrans(p, ' '*len(p)))
#             Convert to lowercase
            content = content.lower()
#             Replace spans of whitespace with single spaces
            content = ' '.join(content.split())
    
#             Add x and y to corresponding data lists
            dataset.append(content)
            labels.append(value)
            if i < 5:
                print(i, content[:200]+'...', '\n')

def encode_char(c):
    try:
        return charlist.index(c)
    except:
        return len(charlist)
        
encoded = []
max_len = max([len(d) for d in dataset])
print(max_len)
for i, d in enumerate(dataset):
    s = ' ' * (max_len - len(d))
    d = d + s
#     try:
    encoded.append(tf.one_hot([encode_char(c) for c in d], len(charlist)+1))
#     except Exception as e:
#         print('Encoding error:')
#         print(d, e, '\n')
    dataset[i] = d


0 once again mr costner has dragged out a movie for far longer than necessary aside from the terrific sea rescue sequences of which there are very few i just did not care about any of the characters mos... 

1 this is an example of why the majority of action films are the same generic and boring there s really nothing worth watching here a complete waste of the then barely tapped talents of ice t and ice cu... 

2 first of all i hate those moronic rappers who could nt act if they had a gun pressed against their foreheads all they do is curse and shoot each other and acting like clichã© e version of gangsters br... 

3 not even the beatles could write songs everyone liked and although walter hill is no mop top he s second to none when it comes to thought provoking action movies the nineties came and social platforms... 

4 brass pictures movies is not a fitting word for them really are somewhat brassy their alluring visual qualities are reminiscent of expensive high class tv commercials

In [79]:
A = np.array(encoded)
encoded = A
print(A.shape, A.size)

labels = np.array(labels)
print(labels.shape)

(200, 5850, 39) 45630000
(200,)


In [87]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(20, input_shape=(5850, 39)),
    tf.keras.layers.Dense(40, activation=tf.keras.activations.tanh),
    tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid)
])
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 20)                4800      
_________________________________________________________________
dense_1 (Dense)              (None, 40)                840       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 41        
Total params: 5,681
Trainable params: 5,681
Non-trainable params: 0
_________________________________________________________________


In [88]:
model.compile(
    optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.01),
    loss=tf.keras.losses.binary_crossentropy,
)

In [89]:
model.fit(encoded, labels, epochs=5, batch_size=16)

Epoch 1/5
13/13 [==============================] - 14s 925ms/step - loss: 7.6515
Epoch 2/5
13/13 [==============================] - 12s 931ms/step - loss: 7.6449
Epoch 3/5
13/13 [==============================] - 12s 927ms/step - loss: 7.6440
Epoch 4/5
13/13 [==============================] - 12s 940ms/step - loss: 3.5429
Epoch 5/5
13/13 [==============================] - 12s 925ms/step - loss: 0.6991


In [91]:
model(encoded[:1])

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.47767258]], dtype=float32)>